In [1]:

import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten	
from keras.layers import Convolution2D, MaxPooling2D
from keras import optimizers
from keras.utils import np_utils
from keras.utils import to_categorical
from pyts.preprocessing import StandardScaler
from pyts.image import GASF, GADF, MTF
import sklearn.metrics as sklm
from sklearn import  metrics
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
#from pandas_ml import ConfusionMatrix
from tensorflow import set_random_seed
from keras.models import load_model
#seeding 1001
seed = 1001
np.random.seed(seed) #for numpy
set_random_seed(seed) #for Tensorflow   

import os

#Read image array from disk
x_train = np.loadtxt('x_train_RP_j.txt').reshape((3496,61,61))

y_train = np.loadtxt('y_train_RP_j.txt')

print(x_train.shape)
print(y_train.shape)



#Normalize the images
def norm(data_x):
#convert to float32 and normalize to [0,1]
    data_x = data_x.astype('float32')
    data_x /= np.amax(data_x)
    return data_x

train_x = norm(x_train)

# convert labels to integers
def l_i(data_y):
    encoder = LabelEncoder()
    encoder.fit(data_y)
    data_y = encoder.transform(data_y)
    return data_y

train_y = l_i(y_train)

print(train_x.shape)
train_x = train_x.reshape(3496,61,61,1)
print(train_x.shape)

#Shufling the data
idx = np.random.permutation(len(train_x))
train_x,train_y = train_x[idx], train_y[idx]

x_test = np.loadtxt('x_test_RP_j.txt').reshape((204,61,61))
x_test = norm(x_test)
x_test = x_test.reshape(204,61,61,1)
print(x_test.shape)

y_test = np.loadtxt('y_test_RP_j.txt')
y_test = l_i(y_test)
print(y_test.shape)


#defining custome metric for guiding our neural network
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    #value, update_op = metrics.roc_auc_score(y_true, y_pred)
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value






/Users/apple/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(3496, 61, 61)
(3496,)
(3496, 61, 61)
(3496, 61, 61, 1)
(204, 61, 61, 1)
(204,)


In [2]:
model2=Sequential()
model2.add(Convolution2D(61, (3,3), padding='same', input_shape=(61,61,1),
                         kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(61, (3,3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Activation('relu'))
model2.add(Convolution2D(32, (3,3), padding='same',kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(32, (3, 3),kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Convolution2D(16, (3,3), padding='same'))
model2.add(Convolution2D(16, (3,3), padding='same'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Convolution2D(16, (3,3), padding='same'))
model2.add(Convolution2D(16, (3,3), padding='same'))
model2.add(Activation('relu'))
model2.add(Convolution2D(16, (3, 3)))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Flatten())
model2.add(Dense(256,kernel_initializer='normal',
                         use_bias=True,bias_initializer='RandomNormal'))
model2.add(Dropout(0.25))
model2.add(Dense(1, activation='sigmoid'))


model2.compile(loss='binary_crossentropy',metrics=[auc_roc],optimizer= optimizers.RMSprop(lr=2e-5))



In [3]:
os.getcwd()

'/Users/apple/Drowsy/60'

In [4]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
filepath="/Users/apple/Drowsy/60/Custom_CNN_weights.h5"

model_checkpoint = ModelCheckpoint(filepath, verbose=1, monitor= 'auc_roc', save_best_only=False, save_weights_only=True)


reduce_lr = ReduceLROnPlateau(monitor= 'auc_roc', patience=100, mode='auto',
                                   cooldown=0, min_lr=1e-4, verbose=2)


callback_list = [model_checkpoint, reduce_lr]


model2.fit(train_x, train_y, epochs=15, verbose=1, callbacks = callback_list) ##fit the model








Epoch 1/15
3496/3496 [==============================] - 267s 76ms/step - loss: 0.6834 - auc_roc: 0.5508

Epoch 00001: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 2/15
3496/3496 [==============================] - 266s 76ms/step - loss: 0.6424 - auc_roc: 0.6263

Epoch 00002: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 3/15
3496/3496 [==============================] - 406s 116ms/step - loss: 0.6106 - auc_roc: 0.6620

Epoch 00003: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 4/15
3496/3496 [==============================] - 264s 76ms/step - loss: 0.5901 - auc_roc: 0.6881

Epoch 00004: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 5/15
3496/3496 [==============================] - 294s 84ms/step - loss: 0.5799 - auc_roc: 0.7052

Epoch 00005: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 6/15
3496/3496 [==============================] - 495s 142ms/step - loss: 0.5649 - auc_roc: 0.7173

In [34]:
loss, auc = model2.evaluate(x_test, y_test)
print('AUC of the model is: ', auc)
print('Loss of the model is: ', loss)

204/204 [==============================] - 7s 33ms/step
AUC of the model is:  0.7879104625944998
Loss of the model is:  0.6486009283393037


In [23]:
yhat = model2.predict_proba(x_test)
roc_mtf=np.column_stack((y_test, yhat))
roc_df=pd.DataFrame(roc_mtf)
roc_df.to_csv("custom_cnn_predicted.csv")

In [35]:
model2.fit(train_x, train_y, epochs=5, verbose=1, callbacks = callback_list) ##fit the model


Epoch 1/5
3496/3496 [==============================] - 267s 76ms/step - loss: 0.4493 - auc_roc: 0.7905

Epoch 00001: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 2/5
3496/3496 [==============================] - 270s 77ms/step - loss: 0.4474 - auc_roc: 0.7964

Epoch 00002: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 3/5
3496/3496 [==============================] - 555s 159ms/step - loss: 0.4290 - auc_roc: 0.8020

Epoch 00003: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 4/5
3496/3496 [==============================] - 265s 76ms/step - loss: 0.4186 - auc_roc: 0.8076

Epoch 00004: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 5/5
3496/3496 [==============================] - 273s 78ms/step - loss: 0.4142 - auc_roc: 0.8127

Epoch 00005: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5


In [36]:
loss, auc = model2.evaluate(x_test, y_test)
print('AUC of the model is: ', auc)
print('Loss of the model is: ', loss)

204/204 [==============================] - 7s 32ms/step
AUC of the model is:  0.8151342635061226
Loss of the model is:  0.6839725112213808


In [44]:
model2.fit(train_x, train_y, epochs=5, verbose=1, callbacks = callback_list, validation_data = (x_test, y_test)) ##fit the model


Train on 3496 samples, validate on 204 samples
Epoch 1/5
3496/3496 [==============================] - 278s 79ms/step - loss: 0.4002 - auc_roc: 0.8174 - val_loss: 0.8792 - val_auc_roc: 0.8195

Epoch 00001: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 2/5
3496/3496 [==============================] - 283s 81ms/step - loss: 0.3886 - auc_roc: 0.8216 - val_loss: 0.6567 - val_auc_roc: 0.8238

Epoch 00002: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 3/5
3496/3496 [==============================] - 275s 79ms/step - loss: 0.3711 - auc_roc: 0.8259 - val_loss: 0.6116 - val_auc_roc: 0.8285

Epoch 00003: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 4/5
3496/3496 [==============================] - 271s 77ms/step - loss: 0.3651 - auc_roc: 0.8307 - val_loss: 0.9605 - val_auc_roc: 0.8326

Epoch 00004: saving model to /Users/apple/Drowsy/60/Custom_CNN_weights.h5
Epoch 5/5
3496/3496 [==============================] - 1377s 394ms/step - los

In [45]:
loss, auc = model2.evaluate(x_test, y_test)
print('AUC of the model is: ', auc)
print('Loss of the model is: ', loss)

204/204 [==============================] - 7s 36ms/step
AUC of the model is:  0.8362144301919376
Loss of the model is:  0.6215098237290102


In [46]:
yhat_3 = model2.predict_proba(x_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, yhat_3)

0.6078584439240177